In [3]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse

# サイトマップ抽出対象のトップページ
base_url = 'https://www.musashino-u.ac.jp/'
visited = set()
site_map = {}
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

def get_title(soup):
    title_tag = soup.find('title')
    return title_tag.text.strip() if title_tag else ''

def crawl(url, base_domain, sleep_sec=1, depth=0, max_depth=2):
    if url in visited:
        return
    if depth > max_depth:  # 再帰の深さを制限
        return
    
    try:
        print(f"アクセス中（深さ{depth}）: {url}")
        res = requests.get(url, headers=headers, timeout=10)
        time.sleep(sleep_sec)
        if res.status_code != 200:
            print(f"ステータスコード: {res.status_code}")
            return
        res.encoding = res.apparent_encoding
        soup = BeautifulSoup(res.text, 'html.parser')
        title = get_title(soup)
        site_map[url] = title
        visited.add(url)
        print(f"✓ 取得成功: {title}")
        
        for a in soup.find_all('a', href=True):
            href = a['href']
            if href.startswith('javascript:') or href.startswith('#') or href.startswith('mailto:'):
                continue
            next_url = urljoin(url, href)
            # URLの正規化（末尾のスラッシュを統一）
            if next_url.endswith('index.html'):
                next_url = next_url.replace('index.html', '')
            
            parsed = urlparse(next_url)
            if parsed.netloc == base_domain and next_url not in visited:
                crawl(next_url, base_domain, sleep_sec, depth + 1, max_depth)
    except requests.exceptions.Timeout:
        print(f"✗ タイムアウト: {url}")
    except requests.exceptions.ConnectionError:
        print(f"✗ 接続エラー: {url}")
    except Exception as e:
        print(f"✗ エラー（{type(e).__name__}）: {url}")

# ドメイン名を抽出
base_domain = urlparse(base_url).netloc
print(f"クローリング開始: {base_domain}")
crawl(base_url, base_domain, sleep_sec=1)

# 結果を表示
print(f"\n=== クローリング完了 ===")
print(f"取得したページ数: {len(site_map)}")
print(site_map)

クローリング開始: www.musashino-u.ac.jp
アクセス中（深さ0）: https://www.musashino-u.ac.jp/
✓ 取得成功: 武蔵野大学
アクセス中（深さ1）: https://www.musashino-u.ac.jp/access.html
✓ 取得成功: 武蔵野大学
アクセス中（深さ1）: https://www.musashino-u.ac.jp/access.html
✓ 取得成功: 交通アクセス | 武蔵野大学
アクセス中（深さ2）: https://www.musashino-u.ac.jp/admission/request.html
✓ 取得成功: 交通アクセス | 武蔵野大学
アクセス中（深さ2）: https://www.musashino-u.ac.jp/admission/request.html
✓ 取得成功: 資料請求 | 入試情報 | 武蔵野大学
アクセス中（深さ2）: https://www.musashino-u.ac.jp/contact.html
✓ 取得成功: 資料請求 | 入試情報 | 武蔵野大学
アクセス中（深さ2）: https://www.musashino-u.ac.jp/contact.html
✓ 取得成功: お問い合わせ | 武蔵野大学
アクセス中（深さ2）: https://www.musashino-u.ac.jp/prospective-students.html
✓ 取得成功: お問い合わせ | 武蔵野大学
アクセス中（深さ2）: https://www.musashino-u.ac.jp/prospective-students.html
✓ 取得成功: 武蔵野大学で学びたい方 | 武蔵野大学
アクセス中（深さ2）: https://www.musashino-u.ac.jp/students.html
✓ 取得成功: 武蔵野大学で学びたい方 | 武蔵野大学
アクセス中（深さ2）: https://www.musashino-u.ac.jp/students.html
✓ 取得成功: 在学生の方 | 武蔵野大学
アクセス中（深さ2）: https://www.musashino-u.ac.jp/alumni.html
✓ 取得成功: 在学生の方 | 武蔵野大学